# Importing of Libraries and reading in of data 

In [242]:
import pandas as pd 

df1 = pd.read_pickle('df_cases_200906.gzip')
df2 = pd.read_pickle('df_label_200906.gzip')

# Sort data by CaseId 

In [243]:
df1 = df1.sort_values('CaseId')
df2 = df2.sort_values('CaseId')

In [244]:
df1.head() 

,CaseId,FileName,Language,StartDate,DocumentType,IsExecuted,OcrText,QualityScore
1161,003061189006,003061189006_69176036_Order form_978-1-62414-9...,EN,20190708,Order form,True,As difficult behavior her myself help.,0.855230
1102,003061189067,003061189067_26173467_Order form_978-1-904782-...,EN,20190530,Order form,True,Present can phone form.,0.849095
1092,003061189156,003061189156_47966765_Contract Documents_978-0...,EN,20190731,Contract Documents,True,Section science difference success wish it wide.,0.894667
1157,003061189229,003061189229_69115288_Master contract_978-0-11...,EN,20190801,Master contract,False,Down maintain Republican.,0.616230
1158,003061189229,003061189229_21094545_other documents_978-1-81...,EN,20190801,other documents,True,Accept sell leader herself if.,0.905182


In [245]:
df2.head() 

,CaseId,label_1,label_2
601,003061189006,True,False
564,003061189067,True,False
558,003061189156,False,False
599,003061189229,False,False
557,003061189242,False,False


Here I drop columns that are not required for the filtering to save on memory

In [246]:
df1 = df1.drop(columns=['Language', 'StartDate', 'DocumentType'])

Converting dataframe to list for the filtering process

In [247]:
cases_list = df1.values.tolist() 
cases_size = len(cases_list)

In [248]:
clean_list = [] 

# Data Filtering

In [249]:
for i in range(0, cases_size): 
    if(cases_list[i][2] == False or cases_list[i][4] < 0.81): 
        cases_list[i][1] = "[" + str(cases_list[i][1]) + "]"
        clean_list.append([cases_list[i][0], cases_list[i][1], "", ""])
    else:
        cases_list[i][1] = "[" + str(cases_list[i][1]) + "]"
        clean_list.append([cases_list[i][0], "", cases_list[i][1], cases_list[i][3]])

# Final prepared dataset dataframe creation

In [250]:
clean_df = pd.DataFrame(clean_list, columns=['CaseId', 'InvalidFileNames', 'ValidFileNames', 'OcrText'])

In [251]:
clean_df.head()

,CaseId,InvalidFileNames,ValidFileNames,OcrText
0,003061189006,,[003061189006_69176036_Order form_978-1-62414-...,As difficult behavior her myself help.
1,003061189067,,[003061189067_26173467_Order form_978-1-904782...,Present can phone form.
2,003061189156,,[003061189156_47966765_Contract Documents_978-...,Section science difference success wish it wide.
3,003061189229,[003061189229_69115288_Master contract_978-0-1...,,
4,003061189229,,[003061189229_21094545_other documents_978-1-8...,Accept sell leader herself if.


Here I am merging rows with the same CaseId  

In [252]:
clean_df = clean_df.groupby(['CaseId'])[['InvalidFileNames', 'ValidFileNames', 'OcrText']].agg(lambda x: ''.join(x.astype(str)))
clean_df.head()

,InvalidFileNames,ValidFileNames,OcrText
CaseId,,,
003061189006,,[003061189006_69176036_Order form_978-1-62414-...,As difficult behavior her myself help.
003061189067,,[003061189067_26173467_Order form_978-1-904782...,Present can phone form.
003061189156,,[003061189156_47966765_Contract Documents_978-...,Section science difference success wish it wide.
003061189229,[003061189229_69115288_Master contract_978-0-1...,[003061189229_21094545_other documents_978-1-8...,Accept sell leader herself if.
003061189242,[003061189242_91658807_other documents_978-0-7...,,


In [253]:
clean_df['InvalidFileNames'] = clean_df['InvalidFileNames'].replace('', '[]')
clean_df['ValidFileNames'] = clean_df['ValidFileNames'].replace('', '[]')
clean_df.head()

,InvalidFileNames,ValidFileNames,OcrText
CaseId,,,
003061189006,[],[003061189006_69176036_Order form_978-1-62414-...,As difficult behavior her myself help.
003061189067,[],[003061189067_26173467_Order form_978-1-904782...,Present can phone form.
003061189156,[],[003061189156_47966765_Contract Documents_978-...,Section science difference success wish it wide.
003061189229,[003061189229_69115288_Master contract_978-0-1...,[003061189229_21094545_other documents_978-1-8...,Accept sell leader herself if.
003061189242,[003061189242_91658807_other documents_978-0-7...,[],


Lastly, I merge the two dataframes with CaseId as the key   

In [254]:
final_df = pd.merge(clean_df, df2, on='CaseId')
final_df.head()

,CaseId,InvalidFileNames,ValidFileNames,OcrText,label_1,label_2
0,003061189006,[],[003061189006_69176036_Order form_978-1-62414-...,As difficult behavior her myself help.,True,False
1,003061189067,[],[003061189067_26173467_Order form_978-1-904782...,Present can phone form.,True,False
2,003061189156,[],[003061189156_47966765_Contract Documents_978-...,Section science difference success wish it wide.,False,False
3,003061189229,[003061189229_69115288_Master contract_978-0-1...,[003061189229_21094545_other documents_978-1-8...,Accept sell leader herself if.,False,False
4,003061189242,[003061189242_91658807_other documents_978-0-7...,[],,False,False


In [255]:
final_df.to_csv("df_final.gzip", index=False, compression="gzip")